# Submiting spark

In [14]:
#!pip install mlrun

In [15]:
# nuclio: ignore
import nuclio

Define the MLRun environment

In [16]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/mlrun"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/mlrun'


In [17]:
# nuclio: start-code

## Function

In [18]:
from mlrun import get_or_create_ctx
from kubernetes import config, client
from kubernetes.stream import stream
import yaml
from pathlib import Path
import os

In [19]:
class K8SClient(object):

    def __init__(self, logger, namespace='default-tenant', config_file=None):
        self.namespace = namespace
        self.logger = logger
        self._init_k8s_config(config_file)
        self.v1api = client.CoreV1Api()

    def _init_k8s_config(self, config_file):
        try:
            config.load_incluster_config()
            self.logger.info('using in-cluster config.')
        except Exception:
            try:
                config.load_kube_config(config_file)
                self.logger.info('using local kubernetes config.')
            except Exception:
                raise RuntimeError(
                    'cannot find local kubernetes config file,'
                    ' place it in ~/.kube/config or specify it in '
                    'KUBECONFIG env var')

    def get_shell_pod_name(self, pod_name='shell'):
        shell_pod = self.v1api.list_namespaced_pod(namespace=self.namespace)
        for i in shell_pod.items:
            if pod_name in i.metadata.name:
                self.logger.info("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))
                shell_name = i.metadata.name
                break
        return shell_name

    def exec_shell_cmd(self, cmd, shell_pod_name = 'shell'):
        shell_name = self.get_shell_pod_name(shell_pod_name)
        # Calling exec and waiting for response
        exec_command = [
            '/bin/bash',
            '-c',
            cmd]
        resp = stream(self.v1api.connect_get_namespaced_pod_exec,
                      shell_name,
                      self.namespace,
                      command=exec_command,
                      stderr=True, stdin=False,
                      stdout=True, tty=False,_preload_content=False)
        
        
        stderr = []
        stdout = []
        while resp.is_open():
            resp.update(timeout=None)
            if resp.peek_stderr():
                 stderr.append(resp.read_stderr())
            if resp.peek_stdout():
                 stdout.append(resp.read_stdout())
           
        err = resp.read_channel(3)
        err = yaml.safe_load(err)
        if err['status'] == 'Success':
            rc = 0
        else:
            rc = int(err['details']['causes'][0]['message'])
        
        stdout_formated=''
        for each in stdout:
            stdout_formated += each
        self.logger.info("STDOUT: %s"% stdout_formated)
        self.logger.info("RC: %s"% rc)
        
        stderr_formated=''
        for each in stderr:
            stderr_formated += each
        if rc != 0:
            self.logger.info("STDERR: %s"% stderr_formated)
            raise Exception('Execution_error', stderr_formated + stdout_formated)
    
def nb_or_py(pyspark_command):
    if pyspark_command.endswith('.ipynb'):
        from mlrun import code_to_function
        dir = Path(pyspark_command).parent
        file =  Path(pyspark_command).stem
        python_file = os.path.join(dir,file +'.py')
        code_to_function(kind="local",filename=pyspark_command,code_output=python_file)
        return python_file
    else:
        return pyspark_command     


def spark_command_builder(name, class_name, jars, packages, spark_options,pyspark_command='',pyspark_params=''):
    cmd = 'spark-submit'
    if name:
        cmd += ' --name ' + name

    if class_name:
        cmd += ' --class ' + class_name

    if jars:
        cmd += ' --jars ' + jars

    if packages:
        cmd += ' --packages ' + packages

    if spark_options:
        cmd += ' ' + spark_options

    if pyspark_command:
        cmd += ' ' + nb_or_py(pyspark_command)
            
    if pyspark_params:
        cmd += ' ' + pyspark_params
        
    return cmd


def spark_submit(context, v3io_access_key, name=None, class_name=None, jars=None, packages=None, spark_options='',pyspark_command='',pyspark_params='',shell_pod_name='shell'):
    """spark_submit function
    
    submiting spark via shell
    
    :param name:        A name of your application.
    :param class_name:  Your application's main class (for Java / Scala apps).
                        * If relative will add to the {artifact_path}
    :param jars:        Comma-separated list of jars to include on the driver
                        and executor classpaths.
    :param packages:    Comma-separated list of maven coordinates of jars to include
                        on the driver and executor classpaths. Will search the local
                        maven repo, then maven central and any additional remote
                        repositories given by --repositories. The format for the
    :param spark_options: spark parametes that are not included as function arguments
    :param pyspark_command: A python script or Jupyter notebook to be executed
    :param pyspark_params:  parameters to the python script

    """
    cmd = spark_command_builder(name, class_name, jars, packages, spark_options,pyspark_command,pyspark_params)
    context.logger.info("submiting :" + cmd)

    cli = K8SClient(context.logger)
    cli.exec_shell_cmd(cmd,shell_pod_name=shell_pod_name)


In [20]:
# nuclio: end-code

## Test
> This test uses the metrics data, created by the [Generator function](https://github.com/mlrun/demo-network-operations/blob/master/notebooks/generator.ipynb) from MLRun's [Network Operations Demo](https://github.com/mlrun/demo-network-operations)  
To test it yourself, please generate this dataset or use any of your available csv/parquet datasets.

In [27]:
import os
from mlrun import code_to_function, mount_v3io, NewTask, mlconf, run_local
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### Define the execute test task

In [28]:
execute_task = NewTask(name='spark-submit',
                         project='submit-proj',
                         params={'pyspark_command':"/User/tmp/test.ipynb", "pyspark_params": "12"},
                         handler=spark_submit)

## Java Example parameters
#params={'spark_options':"/spark/examples/jars/spark-examples_2.11-2.4.4.jar 10",'class_name':'org.apache.spark.examples.SparkPi'},

In [29]:
#submit_run = run_local(submit_task)

### Test on cluster

Convert the code to an MLRun function

In [30]:
fn = code_to_function('spark-submit', handler='spark_submit')

In [31]:
fn.spec.service_account='mlrun-api'
fn.apply(mount_v3io())

In [26]:
fn.export('function.yaml')

> 2020-11-30 14:43:36,470 [info] function spec saved to path: function.yaml


In [ ]:
execute_run = fn.run(execute_task)

### Show results